# Year 2023 Day 16

[--- Day 16: The Floor Will Be Lava ---](https://adventofcode.com/2023/day/16)


## Motive

A video was produced before this notebook, available here: [video problem 202316](/videos/2023/video_problem_202316.html)

This notebook takes a step back and try to visualize with simpler tools, while reusing some code from the video generation (notably the history generation).

## Related puzzles

- [--- Day 14: Parabolic Reflector Dish ---](https://adventofcode.com/2023/day/14)


In [ ]:
from typing import Any, Callable

import numpy as np
import pandas as pd
import xarray as xr
from pyobsplot import Plot

In [ ]:
from advent_of_code.y_2023.problem_202316 import AdventOfCodeProblem202316

problem = AdventOfCodeProblem202316()
problem

In [ ]:
board = xr.DataArray(
    data=problem.parse_input_text_file(),
    dims=("row", "col"),
)
board

In [ ]:
from advent_of_code.visualization.observable_plot import ObservablePlotXarrayBuilder

## With Example Input


In [ ]:
from advent_of_code.common.common import get_example_inputs_file_contents

example_input = get_example_inputs_file_contents(2023)["test_problem_202316"][
    "EXAMPLE_INPUT"
]
parsed_example_input = xr.DataArray(
    problem.parse_text_input(example_input), dims=("row", "col")
)

In [ ]:
from advent_of_code.visualization.observable_plot import ascii_2d_numpy_array_plotter


example_input_plotter = ascii_2d_numpy_array_plotter(
    parsed_example_input,
    obfuscate=False,
    text=True,
    width=300,
    height=300,
)

In [ ]:
example_input_plotter.plot()

We can see the 4 types of mirrors: `-`, `|`, `/` and `\`. Note that a new symbol, `O`, has been introduced. It represented walls. It is a help to manage incoming rays that are outside of the board, and avoid out of bounds coordinates.

Let's get fancy and replace the ASCII characters by Unicode characters, representing more faithfully the mirrors.

See

- [`\` on Wikipédia](https://en.wiktionary.org/wiki/%5C) and [U+FF3C: FULLWIDTH REVERSE SOLIDUS](https://charbase.com/ff3c-unicode-fullwidth-reverse-solidus) (`＼`)
- [`/` on Wikipédia](https://en.wiktionary.org/wiki//) and [U+FF0F: FULLWIDTH SOLIDUS](https://charbase.com/ff0f) (`／`)
- [Box-drawing characters](https://en.wikipedia.org/wiki/Box-drawing_characters) (`─` and `│`)


In [ ]:
parsed_example_input

In [ ]:
from IPython.display import display, Markdown


def to_unicode_board(puzzle_xda: xr.DataArray):
    mapping = {
        "/": "\uff3c",
        "\\": "\uff0f",
        # "-": "\u2500",
        "-": "―",  # best rendered in plot https://www.compart.com/en/unicode/U+2015
        "|": "\u2502",
        # "|": "—",
        ".": "·",
        # "O": "⯀",
        "O": "□",
    }
    unicode_board = puzzle_xda.copy(data=puzzle_xda.values.view("S1").astype("str"))

    # unicode_board[:, 0] = "┃"
    # unicode_board[:, -1] = "┃"
    # unicode_board[0, :] = "━"
    # unicode_board[-1, :] = "━"
    # unicode_board[0, -1] = "┓"
    # unicode_board[-1, 0] = "┗"
    # unicode_board[-1, -1] = "┛"
    # unicode_board[0, 0] = "┏"

    for old, new in mapping.items():
        unicode_board = unicode_board.where(unicode_board != old, new)
    return unicode_board


unicode_board = to_unicode_board(parsed_example_input)
rendered = "\n".join("".join(row) for row in unicode_board.values.tolist())
display(Markdown(f"```\n{rendered}\n```"))

In [ ]:
unicode_board.dtype

In the following, the walls are removed for cleaner visualization.

In [ ]:
unicode_plotter = ascii_2d_numpy_array_plotter(
    unicode_board[1:-1, 1:-1],
    obfuscate=False,
    text=True,
    width=300,
    height=300,
    scheme="Category10",
)
unicode_plotter.plot()

In [ ]:
(unicode_plotter.copy(label=False)
 .unstack(0)
 ).plot()

#### Compute the history

In [ ]:
history_raw = problem.compute_history(parsed_example_input, 250, verbose=False)
history_raw

In [ ]:
len(history_raw)

In [ ]:
history = [
    (depth, tuple(position), tuple(speed)) for (depth, position, speed) in history_raw
]
history

TODO: Use characters to draw the rays? (beware of the 15k actual input...)

In [ ]:
def create_ray_plot_df(history: list[tuple[int, list[int], list[int]]]) -> pd.DataFrame:
    future_df = []
    for depth, pos, speed in history:
        if speed[0] == 0:
            char = "―"
        else:
            char = "\u2502"
        new_entry = (pos[0], pos[1], depth, char)
        future_df.append(new_entry)

    ray_df = pd.DataFrame(future_df, columns=["x", "y", "depth", "char"])
    return ray_df


ray_df = create_ray_plot_df(history)
ray_df

In [ ]:
def plot_rays(board: xr.DataArray, ray_df: pd.DataFrame, *, width: int = 300, 
              dx: int = 12, dy: int = 12):
    plotter = ascii_2d_numpy_array_plotter(
        # board[1:-1, 1:-1],
        board,
        obfuscate=False,
        text=True,
        width=width,
        height=width,
        label=False,
    ).unstack(0)

    x_target = "y"
    y_target = "x"
    plotter.append(
        lambda: [
            Plot.text(  # type:ignore
                ray_df,
                {
                    x_target: "x",
                    y_target: "y",
                    "text": "char",
                    "stroke": "depth",
                    "fill": "currentColor",
                    "dy": dx,
                    "dx": dy,
                    "fontSize": 40 * width / 600 * 13 / board["col"].size,
                },
            )
        ]
    )
    scheme = "turbo"
    scheme = "plasma"
    scheme = "turbo"
    plotter.plot(
        color={"legend": True, "scheme": scheme, "label": "Ray's depth"},
    )


plot_rays(unicode_board, ray_df, width=300)

## With Actual Input


In [ ]:
actual_unicode_board = to_unicode_board(
    xr.DataArray(problem.parse_input_text_file(), dims=("row", "col"))
)
actual_unicode_board

In [ ]:
plot_input = False

In [ ]:
if plot_input:
    unicode_plotter = (
        ascii_2d_numpy_array_plotter(
            actual_unicode_board[1:-1, 1:-1],
            obfuscate=False,
            text=True,
            width=840,
            height=840,
            scheme="Category10",
        )
    )
    unicode_plotter.plot()
    
    unicode_plotter.copy(
        label=False,
        width=840,
        height=840,
    ).unstack(0).plot()


Load History


In [ ]:
import json
from pathlib import Path


actual_history_raw = json.loads(
    (
        Path().resolve()
        / "../../../.."
        / problem.get_visualizations_instructions_for_part_1_file_path()
    )
    .resolve()
    .read_text()
)

In [ ]:
len(actual_history_raw)

Because the history is so long, we must transform it from "raster" to "vector". More precisely, the rays explored the board cell by cell, but not this exploration is done, we can regroup a sequence of consecutive moves in the same direction into a single line, that can be plotted with Observable Plot.

In [ ]:
actual_history = [
    (depth, tuple(position), tuple(speed)) for (depth, position, speed) in actual_history_raw
]
actual_history[:5]

In [ ]:
actual_ray_df = create_ray_plot_df(actual_history)

In [ ]:
plot_rays(actual_unicode_board, actual_ray_df, width=900, dx=5, dy=5)

## Improvements Ideas

The ray do not chain together perfectly well. It is because only their position is considered, and not their direction. Using the direction would help offsetting the unicode characters in a better way, so that the rays really "reflects" on the mirror, with an exact junction. However, this might require more coding, as currently, only the position is taken into account, and this ray's history is flat. Using the full exploration tree might help ; even better, instrumenting the problem's code to produce visualization input material at the same time. Same discussions as described in the notebook 202317 apply here (_during execution_ versus _after execution_ visualization code production). 